## Keras Tuner

신경망은 조정할 하이퍼 파라미터가 굉장히 많습니다.

이를 탐색하는 방법은 

1) Scikeras 를 이용해 scikit-learn 추정기로 변환해 탐색하는 방법
2) 케라스 모델의 하이퍼파라미터 튜닝 라이브러리 KerasTuner 를 사용하는 방법

두 가지가 있습니다.

Keras Tuner를 사용하면 TensorBoard 와도 통합이 가능하므로 아주 유용합니다

사용법을 살펴봅시다

In [8]:
import keras_tuner as kt
import tensorflow as tf
import keras

In [ ]:
keras.optimizers.schedules.ExponentialDecay(decay_steps=)

### 하이퍼파라미터 정의

In [35]:
opt=keras.optimizers.SGD()

def build_model(hp:kt.HyperParameters):
    n_hidden=hp.Int(name="n_hidden",min_value=1,max_value=10,default=2)
    n_units=hp.Int(name="n_units",min_value=64,max_value=128)
    initial_learning_rate=hp.Float(name="initial_lr",min_value=0.001,max_value=0.01)
    decay_steps=hp.Int(name="decay_stapes",min_value=10,max_value=10_00)

    lr_schedule=keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,decay_steps,decay_rate=0.1
    )

    opt=keras.optimizers.SGD(learning_rate=lr_schedule,momentum=0.9,nesterov=True)

    inputs=keras.layers.Input(shape=(9,))
    for i in range(n_hidden):
        if not i:
            hiddens=keras.layers.Dense(
            units=n_units,activation="selu",kernel_initializer="lecun_normal")(inputs)
        else:    
            hiddens=keras.layers.Dense(
                units=n_units,activation="selu",kernel_initializer="lecun_normal")(hiddens)
    outputs=keras.layers.Dense(units=7,activation="sigmoid",kernel_initializer="lecun_normal")(inputs)

    model=keras.Model(inputs,outputs)
    model.compile(optimizer=opt,loss=keras.losses.CategoricalCrossentropy(),
                  metrics=[keras.metrics.CategoricalAccuracy()])
    return model

Keras-Tuner는 오라클을 생성하고

> 오라클 :  하이퍼파라미터 space(즉 조합가능한 모든 하이퍼파라미터 쌍들)을 탐색하여 탐색순서를 정해줌

빈 `Hyperperameters` 객체를 이용해 모델(이때 위의 함수를 호출합니다)을 구축합니다.

### Search 정의

objective : model 이 compile 될때 metrics 중 하나를 참고합니다

#### Random Search

In [48]:
rs_model=kt.RandomSearch(build_model,objective=kt.Objective("val_categorical_accuracy",direction="max"),seed=1)

Reloading Tuner from ./untitled_project/tuner0.json


In [49]:
import keras
import tensorflow as tf
from zipfile import ZipFile
from pandas import read_csv
from pathlib import Path
import io
with ZipFile("../Data/고객 대출등급 분류 해커톤.zip","r") as zpf:
    data=io.BytesIO(zpf.read("고객 대출등급 분류 해커톤/train.csv"))

data=read_csv(data)

import numpy as np
num_data=tf.data.Dataset.from_tensor_slices(data.select_dtypes(np.number))
label_data=tf.data.Dataset.from_tensor_slices(data[["대출등급"]])

# 정규화

normalize=keras.layers.Normalization()
normalize.adapt(num_data)

# 라벨 텐서처리

stringlookup=keras.layers.StringLookup(max_tokens=data["대출등급"].nunique(),num_oov_indices=0)
stringlookup.adapt(label_data)
cate_emb=keras.layers.CategoryEncoding(num_tokens=len(stringlookup.get_vocabulary()))

proceed_label_data=label_data.map(lambda value : 
               cate_emb(stringlookup(value)))

dataset=tf.data.Dataset.zip((num_data,proceed_label_data))

2025-01-20 20:31:38.408248: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-01-20 20:31:53.213896: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [50]:
normalized_dataset=dataset.map(lambda x,y : (tf.squeeze(normalize(x)),y)).repeat(3).shuffle(buffer_size=10_000).batch(16).prefetch(1)

In [51]:
normalized_dataset.element_spec

(TensorSpec(shape=(None, 9), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))

In [52]:
total_size=len(normalized_dataset)
train_size=int(total_size*0.8)
valid_size=int(total_size*0.1)

train=normalized_dataset.take(train_size)
valid=normalized_dataset.skip(train_size).take(valid_size)
test=normalized_dataset.skip(train_size+valid_size)

In [ ]:
rs_model.search(train,epochs=3,validation_data=valid)

위의 결과를 통해 기본적인 랜덤서치 튜닝을 알아보았습니다

#### HyperBand

이번엔 위와 다른 방식으로 하이퍼 모델을 정의하겠습니다

In [ ]:
class ClassificationModel(kt.HyperModel):
    def __init__(self, name=None, tunable=True):
        super().__init__(name, tunable)
        pass
    def build(self, hp:kt.HyperParameters):
        return build_model(hp)
    def fit(self,model:keras.Model,normalize:bool,*args):
        if normalize:
            train=normalized_dataset
        else:
            train=dataset
        return model.fit(train...)

위처럼 서브클래싱을 이용하면 fit에 관련된 변화까지 적용할 수 있습니다.

In [56]:
keras.metrics.CategoricalAccuracy.__name__

'CategoricalAccuracy'

In [57]:
hb_model=kt.Hyperband(build_model,objective="val_categorical_accuracy",factor=3,hyperband_iterations=2)

1. factor

몇번의 에포크 동안 여러 모델을 훈련한 다음, 최악의 모델을 제거하고 상위 1/factor 만큼의 모델만을 남깁니다.

이를 반복하여 단일 모델이 남을떄까지 계속합니다.

2. hyperband_iterations

위의 전체 과정을 hyperband_iterations 만큼 진행합니다

### 참고


은닉층의 개수

> 복잡한 문제일수록 심층 신경망은 파라미터 효율이 좋을 뿐만 아니라, 전이학습의 장접을 가집니다

은닉층의 뉴런 개수

> 스트레치 팬츠

    일반적으로 각 층의 뉴런을 점차 줄이는 깔대기 방식을 선호했으나, 최근엔 모든 은닉층에 같은 개수의 뉴런을 적용합니다.

    이는 여러 장점을 가지는데,

    1) 모든 은닉층의 뉴런 개수가 하나의 하이퍼 파라미터로 나타낼 수 있습니다.

    2) 대부분의 문제에 깔대기 모양의 은닉층과 같거나 더 나은 성능을 보입니다.

    3) 모델에서 문제를 일으키는 병목 층을 피할 수 있습니다.

    "나에게 맞는 사이즈의 바지를 찾는 대신 그냥 큰 스트레치 팬츠를 사서 나중에 알맞게 맞추는 것"과 동일한 개념

> 일반적으로 뉴런수를 늘리는 것보다 층수를 늘리는 것이 더 이점이 많습니다